## **Dim table for Model**

**Creating model dimension table and insrting surrogate key to it.**

In [0]:
from pyspark.sql.functions import *
from delta.tables import *

#Reading data from the silver table

df= spark.sql("select * from ansh_lamba.salesdata.silver")

display(df.limit(7))

**Reading data from source and target model dim table to create surrogate key**

In [0]:
## source table

df_source = spark.sql("select distinct(Model_ID), Model from ansh_lamba.salesdata.silver")
 
display(df_source.limit(5))

In [0]:
# reading data from target table

if spark.catalog.tableExists("ansh_lamba.salesdata.gold"):

    df_target = spark.sql("select Model_ID , Model , Dim_model_key from ansh_lamba.salesdata.gold")

else:

    df_target = spark.sql("select 1 As Dim_model_key, Model_ID, Model from ansh_lamba.salesdata.silver where 0=1")

display(df_target.limit(5))

In [0]:
# reading data from target table

# if spark.catalog.tableExists("ansh_lamba.salesdata.gold"):

#     df_target = spark.sql("select Model_ID , Model , Dim_model_key from ansh_lamba.salesdata.silver")

# else:

#     df_target = spark.sql("select 1 As Dim_model_key, Model_ID, Model from parquet.`abfss://salesdata@adlssouthindiastorage.dfs.core.windows.net/silver/` where 0=1")

# df_target.show()

**Do Left Join to Seggrgate the new and old data**

In [0]:
df_filter = df_source.join(df_target, df_source.Model_ID == df_target.Model_ID , "left").select(df_source.Model_ID, df_source.Model, df_target.Dim_model_key)

display(df_filter.limit(5))

In [0]:
# Old Data from df_filter

df_filter_old = df_filter.filter(df_filter.Dim_model_key.isNotNull())

display(df_filter_old.limit(5))

In [0]:
# New data from df_filer dataframe

df_filter_new = df_filter.filter(df_filter["Dim_model_key"].isNull()).select(col("Model_ID"), col("Model"))

display(df_filter_new.limit(5))

### **Creating Surrogate Key**

**Note - First we need find out maximum value of dim_model_key /**

**for initial load - zero / 
for increamenta load - Max dim_key from target table**

In [0]:
# finding maximun vaue.

if spark.catalog.tableExists("ansh_lamba.salesdata.gold"):

    max_value = spark.sql("select max(Dim_model_key) as max_value from ansh_lamba.salesdata.gold")
    max_value = max_value.collect()[0]['max_value']

    print(f'max value is {max_value}')

else:
    max_value = 1

    print(f'max value is {max_value}')

In [0]:
# Inserting surrogate key to the new data

df_filter_new = df_filter_new.withColumn("Dim_model_key", max_value + monotonically_increasing_id())

display(df_filter_new)

### **Union old and new data**

In [0]:
df_union_new_old = df_filter_old.union(df_filter_new)

display(df_union_new_old)

## **SCD Type 1,  Using upsert increamentally loading into dim model table**

In [0]:
if spark.catalog.tableExists("ansh_lamba.salesdata.gold"):

  df_gold_dim_model_table = DeltaTable.forName(spark, "ansh_lamba.salesdata.gold")

  df_gold_dim_model_table.alias("target") \
    .merge(df_union_new_old.alias("source"), "target.Dim_model_key = source.Dim_model_key") \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

else:

  df_union_new_old.write.mode("overwrite").option("path", "abfss://salesdata@adlssouthindiastorage.dfs.core.windows.net/gold/").saveAsTable("ansh_lamba.salesdata.gold")

In [0]:
%sql
select * from ansh_lamba.salesdata.gold;